
# RE2020 – Générateur de graphiques (Voila)

**Nouveautés v11**
- ✅ Axe X **identique** pour **CEP** et **CEPnr**
- ✅ Option **CEPnr = CEP** (copie toutes les valeurs CEP vers CEPnr, **sauf le max**) + verrouillage des champs CEPnr
- ✅ Notebook **multi-cellules** (imports / widgets / calculs / plots / UI séparés)


In [201]:

import ipywidgets as widgets
from IPython.display import display, FileLink
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [202]:

# === Widgets pour paramètres supplémentaires ===
nbRange_widget = widgets.IntText(description="Nb Range :", value=5)
HeightBar_widget = widgets.IntText(description="Hauteur Bar :", value=6)
heightFond_widget = widgets.IntText(description="Hauteur Fond :", value=18)
graduationx_widget = widgets.Text(
    description="Graduation axe x (bbio, cep, cepnr, icenergie,dh) :",
    value="5,10,10,10,50",
    placeholder="Ex: 5,10,10,10,50"
)

titre_widget = widgets.Textarea(description="Titre :", value="RE2020 - BlaBlaBla le Titre du Projet")
sousTitre_widget = widgets.Textarea(description="Sous-titre :", value="[PHASE] BlaBlaBla l'objectifs machin est atteint")
infos_widget = widgets.Textarea(description="Infos :", value="Projet Machin Machin - Phase XXX - Calcul du XX/XX/20XX - Pléiades version 6.25.X.X - moteur RE 2024.E1.0.0 - WSP France ©")


In [203]:

# === Widgets pour indicateurs ===
BbioChaud_widget = widgets.FloatText(description="Bbio Chaud :", value=1)
BbioFroid_widget = widgets.FloatText(description="Bbio Froid :", value=1)
BbioEcl_widget = widgets.FloatText(description="Bbio Eclairage :", value=1)
Bbiomax_widget = widgets.FloatText(description="Seuil Bbio :", value=10)

CEPch_widget = widgets.FloatText(description="CEP Chauffage :", value=1)
CEPfr_widget = widgets.FloatText(description="CEP Climatisation :", value=1)
CEPecs_widget = widgets.FloatText(description="CEP ECS :", value=1)
CEPecl_widget = widgets.FloatText(description="CEP Éclairage :", value=1)
CEPauxV_widget = widgets.FloatText(description="CEP Auxiliaires de Ventilation :", value=1)
CEPauxD_widget = widgets.FloatText(description="CEP Auxiliaires de Distribution :", value=1)
CEPdep_widget = widgets.FloatText(description="CEP Déplacements :", value=1)
CEPmax_widget = widgets.FloatText(description="Seuil CEP :", value=10)

CEPnrch_widget = widgets.FloatText(description="CEP NR Chauffage :", value=1)
CEPnrfr_widget = widgets.FloatText(description="CEP NR Climatisation :", value=1)
CEPnrecs_widget = widgets.FloatText(description="CEP NR ECS :", value=1)
CEPnrecl_widget = widgets.FloatText(description="CEP NR Éclairage :", value=1)
CEPnrauxV_widget = widgets.FloatText(description="CEP NR Auxiliaires de Ventilation :", value=1)
CEPnrauxD_widget = widgets.FloatText(description="CEP NR Auxiliaires de Distribution :", value=1)
CEPnrdep_widget = widgets.FloatText(description="CEP NR Déplacements :", value=1)
CEPnrmax_widget = widgets.FloatText(description="Seuil CEP NR :", value=10)

# Option: rendre CEPnr = CEP (copie valeurs CEP vers CEPnr, sauf le max)
cepnr_equals_cep_widget = widgets.Checkbox(value=False, description="CEPnr = CEP")

IcEelec_widget = widgets.FloatText(description="IcE Électricité :", value=1)
IcEbois_widget = widgets.FloatText(description="IcE Bois :", value=1)
IcErcu_widget = widgets.FloatText(description="IcE Réseau chaleur/froid :", value=1)
IcEfioul_widget = widgets.FloatText(description="IcE Fioul :", value=1)
IcEgaz_widget = widgets.FloatText(description="IcE Gaz :", value=1)
IcEmax_widget = widgets.FloatText(description="Seuil IcE :", value=10)

DH_widget = widgets.FloatText(description="DH :", value=100)
DHmax_widget = widgets.FloatText(description="Seuil DH :", value=1150)

show_bbio_widget = widgets.Checkbox(value=True, description="Afficher Bbio")
show_cep_widget = widgets.Checkbox(value=True, description="Afficher CEP")
show_cepnr_widget = widgets.Checkbox(value=True, description="Afficher CEPnr")
show_ice_widget = widgets.Checkbox(value=True, description="Afficher IcE")
show_dh_widget = widgets.Checkbox(value=True, description="Afficher DH")


In [204]:

# === Sync CEP -> CEPnr si option activée ===
cep_to_cepnr_map = {
    CEPch_widget: CEPnrch_widget,
    CEPfr_widget: CEPnrfr_widget,
    CEPecs_widget: CEPnrecs_widget,
    CEPecl_widget: CEPnrecl_widget,
    CEPauxV_widget: CEPnrauxV_widget,
    CEPauxD_widget: CEPnrauxD_widget,
    CEPdep_widget: CEPnrdep_widget,
}

def _apply_cep_to_cepnr():
    for w_src, w_dst in cep_to_cepnr_map.items():
        w_dst.value = w_src.value

def _toggle_cepnr_sync(*args):
    enabled = cepnr_equals_cep_widget.value
    for w_dst in cep_to_cepnr_map.values():
        w_dst.disabled = enabled
    if enabled:
        _apply_cep_to_cepnr()

cepnr_equals_cep_widget.observe(lambda ch: _toggle_cepnr_sync(), names='value')

def _on_cep_change(change):
    if cepnr_equals_cep_widget.value:
        for w_src, w_dst in cep_to_cepnr_map.items():
            if change['owner'] is w_src:
                w_dst.value = change['new']
                break

for w_src in cep_to_cepnr_map.keys():
    w_src.observe(_on_cep_change, names='value')

_toggle_cepnr_sync()


In [205]:

# === Helpers calculs ===

def _filter_nonzero(values, labels):
    pairs = [(v, l) for v, l in zip(values, labels) if v != 0]
    if pairs:
        v2, l2 = map(list, zip(*pairs))
        return v2, l2
    return [], []

# === Couleurs (vibes modernes) ===

# Teintes de base (0–360) : change juste ces valeurs si tu veux d’autres ambiances
BASE_HUE = {
    "Bbio": 160,   # teal/vert moderne
    "CEP": 250,    # indigo/violet
    "CEPnr": 270,  # IDENTIQUE à CEP
    "IcE": 35,     # orange/rouge doux
    "DH": 210      # bleu
}


# Fond coloré doux (tinted) par indicateur
# (on utilise des teintes Tailwind très claires + un point un peu plus soutenu) [1](https://wsponline.sharepoint.com/sites/CA-Buildings-BIM/SitePages/Page_ModelAutomation_AddIns_QuickReference_EN.aspx?web=1)[2](https://wsponline.sharepoint.com/sites/GLOBAL-ACC-Guidance/SitePages/Automation-Job-Processor-%28AJP%29-Workflows.aspx?web=1)

BG = {
    "Bbio":  ("#ccfbf1", "#5eead4"),  # teal-100 -> teal-300
    "CEP":   ("#e0e7ff", "#a5b4fc"),  # indigo-100 -> indigo-300
    "CEPnr": ("#e0e7ff", "#a5b4fc"),  # identique CEP
    "IcE":   ("#ffedd5", "#fdba74"),  # orange-100 -> orange-300
    "DH":    ("#e0f2fe", "#7dd3fc"),  # sky-100 -> sky-300
}

def fond_couleur(label, nbRange, reverse=True):
    """
    Dégradé coloré doux (tinted).
    reverse=True => plus foncé à gauche, plus clair à droite (souvent plus joli sur une progression).
    """
    c1, c2 = BG.get(label, ("#f3f4f6", "#d1d5db"))  # fallback gris clair
    cols = sns.blend_palette([c1, c2], n_colors=nbRange)
    return cols[::-1] if reverse else cols



def palette_composants(n, base_h):
    """
    Palette moderne par composant (adaptative au nombre de composantes).
    husl -> couleurs tendances, bien espacées, cohérentes.
    """
    if n <= 0:
        return []
    return sns.husl_palette(n, h=base_h, s=0.75, l=0.55)

# Optionnel (rend les segments plus lisibles)
SEGMENT_EDGE_COLOR = "white"
SEGMENT_EDGE_WIDTH = 0.3
FOND_ALPHA = 0.2


In [206]:
# === Palettes dashboard (Tailwind-ish) ===
# Ordre: du plus foncé au plus clair (pour des segments lisibles)
TW = {
    "teal":  ["#0f766e", "#14b8a6", "#2dd4bf", "#5eead4", "#99f6e4", "#ccfbf1"],  # teal-700..100 [1](https://tailwindcolor.tools/tailwind-colors)
    "indigo":["#4338ca", "#6366f1", "#818cf8", "#a5b4fc", "#c7d2fe", "#e0e7ff"],  # indigo-700..100 [1](https://tailwindcolor.tools/tailwind-colors)
    "orange":["#c2410c", "#f97316", "#fb923c", "#fdba74", "#fed7aa", "#ffedd5"],  # orange-700..100 [1](https://tailwindcolor.tools/tailwind-colors)
    "sky":   ["#0369a1", "#0ea5e9", "#38bdf8", "#7dd3fc", "#bae6fd", "#e0f2fe"],  # sky-700..100 [1](https://tailwindcolor.tools/tailwind-colors)
}

# mapping indicateurs -> famille de couleurs
PALETTE_FAMILY = {
    "Bbio": "teal",
    "CEP": "indigo",
    "CEPnr": "indigo",  # identique CEP
    "IcE": "orange",
    "DH": "sky",
}

def palette_composants(label, n):
    """Retourne n couleurs 'dashboard' pour un indicateur donné."""
    fam = PALETTE_FAMILY.get(label, "indigo")
    base = TW[fam]
    if n <= 0:
        return []
    # si n > 6, on étend doucement en interpolant avec seaborn
    if n <= len(base):
        return base[:n]
    return sns.blend_palette(base, n_colors=n)

In [207]:

# === Calcul des données ===

def calcul_donnees_bbio(nbRange):
    BbioChaud = BbioChaud_widget.value
    BbioFroid = BbioFroid_widget.value
    BbioEcl = BbioEcl_widget.value
    Bbiomax = Bbiomax_widget.value

    bbio = BbioChaud + BbioFroid + BbioEcl

    values = [BbioChaud, BbioFroid, BbioEcl]
    labels = [f"Chauffage : {BbioChaud}", f"Climatisation: {BbioFroid}", f"Eclairage: {BbioEcl}"]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("Bbio", len(values))
    extended_colors = fond_couleur("Bbio", nbRange, reverse=True)


    valuesFond = [Bbiomax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "bbio": bbio,
        "valuesBbio": values,
        "labelsBbio": labels,
        "colorsBbio": colors,
        "extended_colorsBbio": extended_colors,
        "valuesFondBbio": valuesFond,
        "gaucheFondBbio": gaucheFond,
        "Bbiomax": Bbiomax
    }


def calcul_donnees_CEP(nbRange):
    CEPch = CEPch_widget.value
    CEPfr = CEPfr_widget.value
    CEPecs = CEPecs_widget.value
    CEPecl = CEPecl_widget.value
    CEPauxV = CEPauxV_widget.value
    CEPauxD = CEPauxD_widget.value
    CEPdep = CEPdep_widget.value
    CEPmax = CEPmax_widget.value

    CEP = CEPch + CEPfr + CEPecs + CEPecl + CEPauxV + CEPauxD + CEPdep

    values = [CEPch, CEPfr, CEPecs, CEPecl, CEPauxV, CEPauxD, CEPdep]
    labels = [
        f"Chauffage:{CEPch}", f"Climatisation:{CEPfr}", f"ECS:{CEPecs}",
        f"Éclairage:{CEPecl}", f"Aux. ventil.:{CEPauxV}",
        f"Aux. distrib.:{CEPauxD}", f"Déplacements:{CEPdep}"
    ]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("CEP", len(values))
    extended_colors = fond_couleur("CEP", nbRange, reverse=True)
    
    valuesFond = [CEPmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "CEP": CEP,
        "valuesCEP": values,
        "labelsCEP": labels,
        "colorsCEP": colors,
        "extended_colorsCEP": extended_colors,
        "valuesFondCEP": valuesFond,
        "gaucheFondCEP": gaucheFond,
        "CEPmax": CEPmax
    }


def calcul_donnees_CEPnr(nbRange):
    # Si l'option est activée : CEPnr = CEP (sauf max)
    if cepnr_equals_cep_widget.value:
        CEPnrch = CEPch_widget.value
        CEPnrfr = CEPfr_widget.value
        CEPnrecs = CEPecs_widget.value
        CEPnrecl = CEPecl_widget.value
        CEPnrauxV = CEPauxV_widget.value
        CEPnrauxD = CEPauxD_widget.value
        CEPnrdep = CEPdep_widget.value
    else:
        CEPnrch = CEPnrch_widget.value
        CEPnrfr = CEPnrfr_widget.value
        CEPnrecs = CEPnrecs_widget.value
        CEPnrecl = CEPnrecl_widget.value
        CEPnrauxV = CEPnrauxV_widget.value
        CEPnrauxD = CEPnrauxD_widget.value
        CEPnrdep = CEPnrdep_widget.value

    CEPnrmax = CEPnrmax_widget.value

    CEPnr = CEPnrch + CEPnrfr + CEPnrecs + CEPnrecl + CEPnrauxV + CEPnrauxD + CEPnrdep

    values = [CEPnrch, CEPnrfr, CEPnrecs, CEPnrecl, CEPnrauxV, CEPnrauxD, CEPnrdep]
    labels = [
        f"Chauffage:{CEPnrch}", f"Climatisation:{CEPnrfr}", f"ECS:{CEPnrecs}",
        f"Éclairage:{CEPnrecl}", f"Aux. ventil.:{CEPnrauxV}",
        f"Aux. distrib.:{CEPnrauxD}", f"Déplacements:{CEPnrdep}"
    ]
    values, labels = _filter_nonzero(values, labels)

    colors = palette_composants("CEPnr", len(values))  # même famille que CEP
    extended_colors = fond_couleur("CEPnr", nbRange, reverse=True)  # identique CEP
    
    valuesFond = [CEPnrmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "CEPnr": CEPnr,
        "valuesCEPnr": values,
        "labelsCEPnr": labels,
        "colorsCEPnr": colors,
        "extended_colorsCEPnr": extended_colors,
        "valuesFondCEPnr": valuesFond,
        "gaucheFondCEPnr": gaucheFond,
        "CEPnrmax": CEPnrmax
    }


def calcul_donnees_IcE(nbRange):
    IcEelec = IcEelec_widget.value
    IcEbois = IcEbois_widget.value
    IcErcu = IcErcu_widget.value
    IcEfioul = IcEfioul_widget.value
    IcEgaz = IcEgaz_widget.value
    IcEmax = IcEmax_widget.value

    IcE = IcEelec + IcEbois + IcErcu + IcEfioul + IcEgaz

    values = [IcEelec, IcEbois, IcErcu, IcEfioul, IcEgaz]
    labels = [f"Elec:{IcEelec}", f"Bois:{IcEbois}", f"RCU:{IcErcu}", f"Fioul:{IcEfioul}", f"Gaz:{IcEgaz}"]
    values, labels = _filter_nonzero(values, labels)
    
    colors = palette_composants("IcE", len(values))
    extended_colors = fond_couleur("IcE", nbRange, reverse=True)

    valuesFond = [IcEmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "IcE": IcE,
        "valuesIcE": values,
        "labelsIcE": labels,
        "colorsIcE": colors,
        "extended_colorsIcE": extended_colors,
        "valuesFondIcE": valuesFond,
        "gaucheFondIcE": gaucheFond,
        "IcEmax": IcEmax
    }


def calcul_donnees_DH(nbRange):
    DH = DH_widget.value
    DHmax = DHmax_widget.value

    values = [DH]
    labels = [f"Degré heure : {DH}"]
    
    colors = palette_composants("DH", len(values))
    extended_colors = fond_couleur("DH", nbRange, reverse=True)

    valuesFond = [DHmax / nbRange] * nbRange
    gaucheFond = np.cumsum(valuesFond) - valuesFond[0]

    return {
        "DH": DH,
        "valuesDH": values,
        "labelsDH": labels,
        "colorsDH": colors,
        "extended_colorsDH": extended_colors,
        "valuesFondDH": valuesFond,
        "gaucheFondDH": gaucheFond,
        "DHmax": DHmax
    }


In [218]:

# === Affichage des graphes (layout spacers + CEP/CEPnr axe X commun) ===

def afficher_graphique(d_bbio, d_CEP, d_CEPnr, d_IcE, d_DH,
                       nbRange, HeightBar, heightFond, graduationx,
                       titre, sousTitre, infos):

    selected = []
    if show_bbio_widget.value: selected.append(("Bbio", d_bbio, graduationx[0]))
    if show_cep_widget.value: selected.append(("CEP", d_CEP, graduationx[1]))
    if show_cepnr_widget.value: selected.append(("CEPnr", d_CEPnr, graduationx[2]))
    if show_ice_widget.value: selected.append(("IcE", d_IcE, graduationx[3]))
    if show_dh_widget.value: selected.append(("DH", d_DH, graduationx[4]))

    n = len(selected)
    if n == 0:
        print("Aucun indicateur sélectionné")
        return

    # Axe X commun CEP/CEPnr
    max_raw_cep = max(d_CEP.get('CEPmax', 0), d_CEP.get('CEP', 0), d_CEPnr.get('CEPnrmax', 0), d_CEPnr.get('CEPnr', 0))
    grad_common = min(graduationx[1], graduationx[2])
    xlim_cep = 1.05 * max_raw_cep
    xticks_cep = np.arange(0, 1 + max_raw_cep, grad_common)

    fig_w = 30/2.54
    fig_h = 2.0 + 1.65 * n
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=100)

    header_title_r = 0.38
    header_sub_r   = 0.22
    footer_r       = 0.45
    title_r        = 0.20

    legend_rs = []
    for (label, data, grad) in selected:
        m = len(data.get(f"labels{label}", []))
        if m == 0: legend_rs.append(0.12)
        elif m <= 3: legend_rs.append(0.18)
        elif m <= 6: legend_rs.append(0.24)
        elif m <= 10: legend_rs.append(0.34)
        else: legend_rs.append(0.45)

    spacer_between = 0.35
    spacer_before_footer = 0.32 

    height_ratios = [header_title_r, header_sub_r]
    for i, lr in enumerate(legend_rs):
        height_ratios.extend([title_r, lr, 1.0])
        height_ratios.append(spacer_before_footer if i == n-1 else spacer_between)
    height_ratios.append(footer_r)

    total_rows = 2 + 4*n + 1
    gs = fig.add_gridspec(nrows=total_rows, ncols=1, height_ratios=height_ratios, hspace=0.03)

    ax_title = fig.add_subplot(gs[0, 0]); ax_title.axis('off')
    ax_sub   = fig.add_subplot(gs[1, 0]); ax_sub.axis('off')
    ax_title.text(0.0, 0.05, titre, ha='left', va='bottom', fontweight='bold', fontsize=16)
    ax_sub.text(0.0, 0.95, sousTitre, ha='left', va='top', fontsize=11) #ICICICICICICICICICIICICICICICICICICICICICIICIC

    for idx, (label, data, grad) in enumerate(selected):
        base_row = 2 + 4*idx
        ax_t = fig.add_subplot(gs[base_row, 0]); ax_t.axis('off')
        ax_l = fig.add_subplot(gs[base_row+1, 0]); ax_l.axis('off')
        ax   = fig.add_subplot(gs[base_row+2, 0])
        ax_s = fig.add_subplot(gs[base_row+3, 0]); ax_s.axis('off')

        ax_t.text(0.0, 0.5, label, ha='left', va='center', fontweight='bold', fontsize=12)


        ax.barh(
            1,
            data[f"valuesFond{label}"],
            left=data[f"gaucheFond{label}"],
            height=heightFond,
            color=data[f"extended_colors{label}"],
            alpha=FOND_ALPHA,
            edgecolor="none"
        )


        left_pos = 0
        for i, val in enumerate(data[f"values{label}"]):
            ax.barh(
                1,
                val,
                color=data[f"colors{label}"][i],
                left=left_pos,
                height=HeightBar,
                label=data[f"labels{label}"][i],
                zorder=3,
                edgecolor=SEGMENT_EDGE_COLOR,
                linewidth=SEGMENT_EDGE_WIDTH
            )
            left_pos += val

        ax.axvline(data[f"{label}max"], color="black", linewidth=3)
        ax.text(1.01 * data[f"{label}max"], 1,
                f"{label} max{data[f'{label}max']}",
                fontsize=7.5, va="center")

        if label in ("CEP", "CEPnr"):
            ax.set_xlim(0, xlim_cep)
            ax.set_xticks(xticks_cep)
        else:
            # clé valeur totale
            key_tot = {"Bbio": "bbio", "IcE": "IcE", "DH": "DH"}.get(label, label)
            valeur = data.get(key_tot, 0)
            seuil = data.get(f"{label}max", 0)
            ax.set_xlim(0, 1.05 * max(seuil, valeur))
            ax.set_xticks(np.arange(0, 1 + max(seuil, valeur), grad))

        ax.set_yticks([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.tick_params(axis='x', labelsize=8, pad=1)

        handles, labels_txt = ax.get_legend_handles_labels()
        if labels_txt:
            m = len(labels_txt)
            ncol = m if m <= 6 else int(np.ceil(m/2))
            ncol = max(1, min(ncol, 6))
            fontsize = 8.5 if m <= 6 else 8
            ax_l.legend(handles, labels_txt, loc='center left', ncol=ncol,
                        frameon=False, fontsize=fontsize,
                        handlelength=0.8, handletextpad=0.35,
                        columnspacing=0.8, borderaxespad=0.0)

    ax_footer = fig.add_subplot(gs[-1, 0]); ax_footer.axis('off')
    ax_footer.text(0.0, 0.55, infos, ha='left', va='center', fontsize=8, style='italic')

    fig.savefig(f"{titre}_Graph.png", dpi=250)
    plt.show()
    display(FileLink(f"{titre}_Graph.png"))


In [219]:

# === UI (Voila) ===

bouton_graph = widgets.Button(description="Mettre à jour le graphique", button_style='success', layout=widgets.Layout(width='240px'))
output_graph = widgets.Output()

def on_button_click(b):
    with output_graph:
        output_graph.clear_output(wait=True)

        nbRange = nbRange_widget.value
        HeightBar = HeightBar_widget.value
        heightFond = heightFond_widget.value
        titre = titre_widget.value
        sousTitre = sousTitre_widget.value
        infos = infos_widget.value
        graduationx = [int(x.strip()) for x in graduationx_widget.value.split(",")]

        d_bbio = calcul_donnees_bbio(nbRange)
        d_CEP = calcul_donnees_CEP(nbRange)
        d_CEPnr = calcul_donnees_CEPnr(nbRange)
        d_IcE = calcul_donnees_IcE(nbRange)
        d_DH = calcul_donnees_DH(nbRange)

        afficher_graphique(d_bbio, d_CEP, d_CEPnr, d_IcE, d_DH,
                          nbRange, HeightBar, heightFond, graduationx,
                          titre, sousTitre, infos)

bouton_graph.on_click(on_button_click)

# Style global
style_desc = {'description_width': '200px'}
large_layout = widgets.Layout(width='420px')

all_widgets = [
    BbioChaud_widget, BbioFroid_widget, BbioEcl_widget, Bbiomax_widget,
    CEPch_widget, CEPfr_widget, CEPecs_widget, CEPecl_widget, CEPauxV_widget,
    CEPauxD_widget, CEPdep_widget, CEPmax_widget,
    CEPnrch_widget, CEPnrfr_widget, CEPnrecs_widget, CEPnrecl_widget,
    CEPnrauxV_widget, CEPnrauxD_widget, CEPnrdep_widget, CEPnrmax_widget,
    cepnr_equals_cep_widget,
    IcEelec_widget, IcEbois_widget, IcErcu_widget, IcEfioul_widget,
    IcEgaz_widget, IcEmax_widget,
    DH_widget, DHmax_widget,
    titre_widget, sousTitre_widget, infos_widget,
    nbRange_widget, HeightBar_widget, heightFond_widget, graduationx_widget
]

for w in all_widgets:
    w.layout = large_layout
    w.style = style_desc

section_info = widgets.VBox([
    widgets.HTML("<h2>Informations générales</h2>"),
    titre_widget, sousTitre_widget, infos_widget
])

section_bbio = widgets.VBox([
    widgets.HTML("<h2>Valeurs Bbio</h2>"),
    BbioChaud_widget, BbioFroid_widget, BbioEcl_widget, Bbiomax_widget, show_bbio_widget
])

section_cep_group = widgets.VBox([
    widgets.HTML("<h2>Valeurs CEP</h2>"),
    CEPch_widget, CEPfr_widget, CEPecs_widget, CEPecl_widget,
    CEPauxV_widget, CEPauxD_widget, CEPdep_widget, CEPmax_widget, show_cep_widget,
    widgets.HTML("<h2>Valeurs CEPnr</h2>"),
    CEPnrch_widget, CEPnrfr_widget, CEPnrecs_widget, CEPnrecl_widget,
    CEPnrauxV_widget, CEPnrauxD_widget, CEPnrdep_widget, CEPnrmax_widget,
    show_cepnr_widget,
    cepnr_equals_cep_widget
])

section_ice = widgets.VBox([
    widgets.HTML("<h2>Valeurs IcE</h2>"),
    IcEelec_widget, IcEbois_widget, IcErcu_widget, IcEfioul_widget,
    IcEgaz_widget, IcEmax_widget, show_ice_widget
])

section_dh = widgets.VBox([
    widgets.HTML("<h2>Valeurs DH</h2>"),
    DH_widget, DHmax_widget, show_dh_widget
])

section_graph_params = widgets.VBox([
    widgets.HTML("<h2>Paramètres graphiques</h2>"),
    nbRange_widget, HeightBar_widget, heightFond_widget, graduationx_widget
])

accordion = widgets.Accordion(children=[
    section_info, section_bbio, section_cep_group, section_ice, section_dh, section_graph_params
])
accordion.set_title(0, 'Informations générales')
accordion.set_title(1, 'Valeurs Bbio')
accordion.set_title(2, 'Valeurs CEP & CEPnr')
accordion.set_title(3, 'Valeurs IcE')
accordion.set_title(4, 'Valeurs DH')
accordion.set_title(5, 'Paramètres graphiques')

ui = widgets.VBox([accordion, bouton_graph, output_graph])
display(ui)
